<a href="https://colab.research.google.com/github/mzohaibnasir/NeuralNotes/blob/main/03_deepDiveIntoBasics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minibatch Training

In [160]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl,numpy as np,matplotlib.pyplot as plt
from pathlib import Path
from torch import tensor,nn
import torch.nn.functional as F
from fastcore.test import test_close


In [161]:
MNIST_URL='https://github.com/mnielsen/neural-networks-and-deep-learning/blob/d15df08a69ed33ae16a2fff874f83b57a956172c/data/mnist.pkl.gz?raw=true'
path_data = Path('data')
path_data.mkdir(exist_ok=True )
path_gz=path_data/'mnist.pkl.gz'
path_gz

PosixPath('data/mnist.pkl.gz')

In [162]:
from urllib.request import urlretrieve

if not path_gz.exists(): urlretrieve(MNIST_URL, path_gz)

In [163]:
torch.manual_seed(42)


mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)


In [164]:
!ls -l data

total 16656
-rw-r--r-- 1 root root 17051982 Jan 20 14:02 mnist.pkl.gz


In [165]:
with gzip.open(path_gz, 'rb') as f:   #read as binary as opposed to text
   ((x_train,y_train), (x_valid,y_valid), _) = pickle.load(f, encoding='latin-1') #destructuring
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

In [166]:
n,m = x_train.shape
c = y_train.max()+1
nh = 50


In [167]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]

    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x

In [168]:
model = Model(m, nh, 10)
pred = model(x_train)
pred.shape

torch.Size([50000, 10])

In [169]:
### CE loss

In [170]:
def log_softmax(x):
  print(x.shape)
  return (x.exp()/x.exp().sum(-1, keepdim=True)).shape
log_softmax(pred)

torch.Size([50000, 10])


torch.Size([50000, 10])

In [171]:
def log_softmax(x):
  # print(x.shape)
  return (x.exp()/x.exp().sum(-1, keepdim=True)).log()
log_softmax(pred)

tensor([[-2.36, -2.28, -2.09,  ..., -2.43, -2.47, -2.11],
        [-2.37, -2.25, -2.09,  ..., -2.46, -2.43, -2.11],
        [-2.34, -2.31, -2.14,  ..., -2.44, -2.48, -2.14],
        ...,
        [-2.26, -2.25, -2.13,  ..., -2.36, -2.53, -2.17],
        [-2.39, -2.30, -2.18,  ..., -2.38, -2.42, -2.11],
        [-2.40, -2.25, -2.14,  ..., -2.38, -2.45, -2.23]], grad_fn=<LogBackward0>)

In [172]:
pred.exp().shape , pred.exp().sum(-1)[:,None].shape

(torch.Size([50000, 10]), torch.Size([50000, 1]))

In [173]:
pred.shape

torch.Size([50000, 10])

we'll try to use log rules to simply softmax funcction

`e^x and log x are opposite`

In [174]:
def log_softmax(x):
  return x - x.exp().sum(-1, keepdim=True).log()

In [175]:
def logsumexp(x):
  m = x.max(-1)[0]  # max value of `x`
  return m + (x-m[:,None]).exp().sum(-1).log()  #logsumexp trick

In [176]:
# in pytorch

def log_softmax(x):
  return x - x.logsumexp(-1, keepdim=True)

In [177]:
test_close( logsumexp(pred), pred.logsumexp(-1))
sm_pred = log_softmax(pred)
sm_pred

tensor([[-2.36, -2.28, -2.09,  ..., -2.43, -2.47, -2.11],
        [-2.37, -2.25, -2.09,  ..., -2.46, -2.43, -2.11],
        [-2.34, -2.31, -2.14,  ..., -2.44, -2.48, -2.14],
        ...,
        [-2.26, -2.25, -2.13,  ..., -2.36, -2.53, -2.17],
        [-2.39, -2.30, -2.18,  ..., -2.38, -2.42, -2.11],
        [-2.40, -2.25, -2.14,  ..., -2.38, -2.45, -2.23]], grad_fn=<SubBackward0>)

The cross entropy loss for some target
 and some prediction  p(x) is given by:
      - sum( x . log(p(x)))

  But since our xs are 1-hot encoded (actually, they're just the integer indices), this can be rewritten as -log(pi) where i is the index of the desired target.

This can be done using numpy-style integer array indexing. Note that PyTorch supports all the tricks in the advanced indexing methods discussed in that link.

In [178]:
y_train[:3]  # actual values of y_train

tensor([5, 0, 4])

In [179]:
# now in our softmaax prediction `sm_pred`, we want to get 5th prediction of 0, 0th of 1, and 4th of 2

sm_pred[0,5], sm_pred[1,0], sm_pred[2,4]

(tensor(-2.40, grad_fn=<SelectBackward0>),
 tensor(-2.37, grad_fn=<SelectBackward0>),
 tensor(-2.14, grad_fn=<SelectBackward0>))

In [180]:
sm_pred.shape

torch.Size([50000, 10])

In [181]:
sm_pred[[0,1,2], y_train[:3]]     # [rows, cols]

tensor([-2.40, -2.37, -2.14], grad_fn=<IndexBackward0>)

In [182]:
sm_pred[range(y_train.shape[0])]

tensor([[-2.36, -2.28, -2.09,  ..., -2.43, -2.47, -2.11],
        [-2.37, -2.25, -2.09,  ..., -2.46, -2.43, -2.11],
        [-2.34, -2.31, -2.14,  ..., -2.44, -2.48, -2.14],
        ...,
        [-2.26, -2.25, -2.13,  ..., -2.36, -2.53, -2.17],
        [-2.39, -2.30, -2.18,  ..., -2.38, -2.42, -2.11],
        [-2.40, -2.25, -2.14,  ..., -2.38, -2.45, -2.23]], grad_fn=<IndexBackward0>)

In [183]:
def nll(input, target): # this will cross entropy loss   # negative non-likelihood loss
  return -input[range(target.shape[0]), target].mean()

In [184]:
loss = nll(sm_pred, y_train)
loss

tensor(2.30, grad_fn=<NegBackward0>)

In [185]:
# Then use PyTorch's implementation.
test_close(F.nll_loss(F.log_softmax(pred, -1), y_train), loss, 1e-3)





In [186]:
# In PyTorch, F.log_softmax and F.nll_loss are combined in one optimized function, F.cross_entropy.
test_close(F.cross_entropy(pred, y_train), loss, 1e-3)


# basic training loop

Basic training loop
Basically the training loop repeats over the following steps:

 * get the output of the model on a batch of inputs
 * compare the output to the labels we have and compute a loss
 * calculate the gradients of the loss with respect to every parameter of the model
 * update said parameters with those gradients to make them a little bit better

In [187]:
loss_func = F.cross_entropy

In [188]:
bs = 50  # batch size
xb = x_train[0:bs]  # a mini batch from x
preds = model(xb)

preds[0], preds.shape

(tensor([-0.05,  0.03,  0.22,  0.02,  0.00, -0.09, -0.04, -0.12, -0.15,  0.20], grad_fn=<SelectBackward0>),
 torch.Size([50, 10]))

In [189]:
yb = y_train[0:bs]
yb

tensor([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0, 9, 1, 1, 2, 4, 3, 2, 7, 3, 8, 6, 9, 0, 5, 6, 0, 7,
        6, 1, 8, 7, 9, 3, 9, 8, 5, 9, 3])

In [190]:
loss_func(preds, yb)

tensor(2.28, grad_fn=<NllLossBackward0>)

In [191]:
preds.argmax(dim=1) # for each rows in preds , find highest number

tensor([2, 2, 4, 2, 2, 9, 2, 2, 1, 9, 9, 9, 2, 2, 1, 2, 2, 2, 9, 9, 2, 2, 9, 2, 2, 2, 2, 2, 2, 1, 9, 1, 2, 2, 2, 9, 2, 9, 2,
        9, 9, 2, 9, 2, 9, 2, 2, 2, 2, 2])

In [192]:
def accuracy(out, yb):
  return (out.argmax(dim = 1) == yb).float().mean()

In [193]:
accuracy(preds, yb)

tensor(0.16)

In [194]:
lr = 0.5
epochs = 3

In [195]:
def report(loss, preds, yb):
  print(f"{loss:.2f}, {accuracy(preds,yb):.2f}")

In [196]:
xb,yb = x_train[:bs],y_train[:bs]
preds = model(xb)
report(loss_func(preds, yb), preds, yb)

2.28, 0.16


In [197]:
for epoch in range(epochs):
  for i in range(0, n, bs):
    s = slice(i, min(n, i+bs))
    xb, yb = x_train[s], y_train[s]
    preds = model(xb)
    loss = loss_func(preds, yb)
    loss.backward()

    with torch.no_grad():
      for l in model.layers:
        if( hasattr( l , 'weight')):
          l.weight -= l.weight.grad * lr
          l.bias -= l.bias.grad * lr

          l.weight.grad.zero_()
          l.bias.grad.zero_()

  report(loss, preds, yb)

0.08, 0.96
0.05, 0.98
0.07, 0.96


# Using parameters and optim


## parameters

In [198]:
m1 = nn.Module()
m1, list(m1.parameters())

(Module(), [])

In [199]:
m1.foo = nn.Linear(3,4)  #  the number of neurons in a linear layer is determined by the number of output features.
m1

Module(
  (foo): Linear(in_features=3, out_features=4, bias=True)
)

In [200]:
list(m1.named_children())

[('foo', Linear(in_features=3, out_features=4, bias=True))]

In [201]:
m1.named_children()

<generator object Module.named_children at 0x7b62d43231b0>

In [202]:
list(m1.parameters())

[Parameter containing:
 tensor([[-0.24, -0.11,  0.46],
         [-0.17, -0.46, -0.07],
         [-0.29,  0.31,  0.08],
         [-0.24,  0.13, -0.17]], requires_grad=True),
 Parameter containing:
 tensor([ 0.01,  0.51, -0.11, -0.47], requires_grad=True)]

In [203]:
# multi layer perceptron

class MLP(nn.Module):
  def __init__(self, n_in, nh, n_out):
    super().__init__()
    self.l1 = nn.Linear(n_in,nh )
    self.l2 = nn.Linear(nh, n_out)
    self.relu = nn.ReLU()


  def __call__(self, x):
    return self.l2(self.relu(self.l1(x)))

In [204]:
model = MLP(m, nh, 10)
model

MLP(
  (l1): Linear(in_features=784, out_features=50, bias=True)
  (l2): Linear(in_features=50, out_features=10, bias=True)
  (relu): ReLU()
)

In [205]:
model.l1

Linear(in_features=784, out_features=50, bias=True)

In [206]:
for name, l in model.named_children():
  print(f"{name}: {l}")

l1: Linear(in_features=784, out_features=50, bias=True)
l2: Linear(in_features=50, out_features=10, bias=True)
relu: ReLU()


In [207]:
for p in model.parameters():   # parameters are `weights` + `bias`
  print(p.shape)


torch.Size([50, 784])
torch.Size([50])
torch.Size([10, 50])
torch.Size([10])


In [208]:
[l for l in model.children()]

[Linear(in_features=784, out_features=50, bias=True),
 Linear(in_features=50, out_features=10, bias=True),
 ReLU()]

In [209]:
"""
torch.no_grad() is used to perform parameter updates without tracking gradients during this specific block of code.
This is a common practice during the training loop when you have parts that involve updating the model parameters but don't need to be included
in the gradient computation.

"""
def fit():
  for epoch in range(epochs):
    for i in range(0, n, bs):
      s = slice(i , min(n, i+bs))
      xb, yb = x_train[s], y_train[s]
      preds = model(xb)

      loss = loss_func(preds, yb)
      loss.backward()  # it just computes gradients and stores gradients in `.grad`
      with torch.no_grad():
        for p in model.parameters():
          p-= p.grad * lr   # manually updating parameters
        model.zero_grad()  # to zero all gradients
    report(loss, preds, yb)


    """
    but how does it know what paarameters and layers are automatically? it used a trick called __setattr__

    """

fit()

0.12, 0.96
0.11, 0.98
0.09, 0.96


Behind the scenes, PyTorch overrides the __setattr__ function in nn.Module so that the submodules you define are properly registered as parameters of the model.



In [210]:
#     but how does it know what paarameters and layers are automatically? it used a trick called __setattr__

class MyModule:     # its by defult class MyModule(object)
  def __init__(self, n_in, nh, n_out):
    self._modules = {}
    self.l1 = nn.Linear(n_in, nh)
    self.l2 = nn.Linear(nh, n_out)


  def __setattr__(self,k,v):
        if not k.startswith("_"): self._modules[k] = v  # to make sure its private  # it adds that attr in modules directory
        super().__setattr__(k,v)

        # every attibute you set like     self.l1 = nn.Linear(n_in, nh) , it

  def __repr__(self): return f'{self._modules}'   # how it represents modules dic

  def parameters(self):
      for l in self._modules.values():   # go through each of modules.  value of modules is actual layers
        yield from l.parameters()

In [211]:
mdl = MyModule(m,nh,10)
mdl

{'l1': Linear(in_features=784, out_features=50, bias=True), 'l2': Linear(in_features=50, out_features=10, bias=True)}

In [212]:
mdl._modules

{'l1': Linear(in_features=784, out_features=50, bias=True),
 'l2': Linear(in_features=50, out_features=10, bias=True)}

In [213]:
for p in mdl.parameters(): print(p.shape)


torch.Size([50, 784])
torch.Size([50])
torch.Size([10, 50])
torch.Size([10])


In [214]:
def numbers_to_10():
    for i in range(1, 11):
        yield i

def numbers_to_20():
    yield from numbers_to_10()  # Yield values from another generator()..   if we dont use yield, values would be skipped
    for i in range(11, 21):
        yield i

# for num in numbers_to_20():
#     print(num)  # Outputs numbers 1 to 20

list(numbers_to_20())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [215]:
list(numbers_to_10())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [216]:
numbers_to_10()

<generator object numbers_to_10 at 0x7b62d4064a50>

# Registering modules

In [217]:
from functools import reduce

We can use the original layers approach, but we have to register the modules.



In [218]:
layers = [  nn.Linear(m,nh),
            nn.ReLU(),
            nn.Linear(nh, 10)]
layers

[Linear(in_features=784, out_features=50, bias=True),
 ReLU(),
 Linear(in_features=50, out_features=10, bias=True)]

In [219]:
from functools import reduce


In [220]:
class Model(nn.Module):
  def __init__(self, layers):
    super().__init__()
    self.layers = layers
    for i,l in enumerate(self.layers):
      self.add_module(f'layer_{i}', l)

  def forward(self, x):
    return reduce(lambda val, layer: layer(val), self.layers, x)  # reduce(reduction_function, self.layers, x)


"""result = reduce(reduction_function, self.layers, x)
The reduce function is called with three arguments:

reduction_function: The binary function to apply cumulatively.
self.layers: The iterable (list of layers) to apply the function to.
x: The initial value to start the accumulation.
The reduce function iterates through the layers, applying the lambda function to the accumulated value (val) and the current layer, updating the accumulated
value at each step. The final result is the output of the last layer applied to the initial input x."""

'result = reduce(reduction_function, self.layers, x)\nThe reduce function is called with three arguments:\n\nreduction_function: The binary function to apply cumulatively.\nself.layers: The iterable (list of layers) to apply the function to.\nx: The initial value to start the accumulation.\nThe reduce function iterates through the layers, applying the lambda function to the accumulated value (val) and the current layer, updating the accumulated \nvalue at each step. The final result is the output of the last layer applied to the initial input x.'

In [221]:
model = Model(layers)
model

Model(
  (layer_0): Linear(in_features=784, out_features=50, bias=True)
  (layer_1): ReLU()
  (layer_2): Linear(in_features=50, out_features=10, bias=True)
)

In [222]:
s= [ 1,2,3]

reduce( lambda x,y : x+y,s , 10)

16

In [223]:
model(xb).shape

torch.Size([50, 10])

# nn.ModuleList
  nn.ModuleList does this for us

In [224]:
class SequentialModel(nn.Module):
  def __init__(self, layers):
    super().__init__()
    self.layers = nn.ModuleList(layers)# replaces `self.layers = layers; for i,l in enumerate(self.layers): self.add_module(f'layer_{i}', l)

  # def forward(self,x):
  #   for l in self.layers:x = l(x);
  #   return x

  def forward(self,x):
    return reduce( lambda v,l :l(v), self.layers, x )

In [225]:
model = SequentialModel(layers)
model

SequentialModel(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)

In [226]:
fit()

0.13, 0.92
0.10, 0.96
0.08, 0.96


In [227]:
x = [1,2,3,4,5,6]
list(filter(lambda x : x%2 != 0 , x))

[1, 3, 5]

# nn.Sequential
nn.Sequential does same by default


In [228]:
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))
model


Sequential(
  (0): Linear(in_features=784, out_features=50, bias=True)
  (1): ReLU()
  (2): Linear(in_features=50, out_features=10, bias=True)
)

In [229]:
fit()

0.16, 0.96
0.10, 0.96
0.05, 0.98


In [230]:
loss_func(model(xb), yb),      accuracy(model(xb), yb)


(tensor(0.02, grad_fn=<NllLossBackward0>), tensor(1.))

# optim

In [231]:
class Optimizer:
  def __init__(self, params, lr = 0.5):
    self.params, self.lr = list(params), lr

  def step(self):
    with torch.no_grad():
      for p in self.params:
        p -= p.grad * self.lr

  def zero_grad(self):
    for p in self.params:
      p.grad.data.zero_()


In [232]:
model = nn.Sequential(
    nn.Linear(m,nh),
    nn.ReLU(),
    nn.Linear(nh, 10)
)

In [233]:
opt = Optimizer(model.parameters())

In [234]:
for epoch in range(epochs):
  for i in range(0 , n, bs):
    s = slice(i, min(i+bs , n))

    xb,yb = x_train[s], y_train[s]
    preds = model(xb)

    loss = loss_func(preds, yb)
    loss.backward()  # computes gradients

    opt.step()
    opt.zero_grad()
  report(loss, preds, yb)

0.10, 0.98
0.06, 1.00
0.03, 1.00


PyTorch already provides this exact functionality in optim.SGD (it also handles stuff like momentum, which we'll look at later)

In [235]:
from torch import optim

In [236]:
def get_model():
  model = nn.Sequential(
      nn.Linear(m,nh),
      nn.ReLU(),
      nn.Linear(nh, 10)
  )
  return model, optim.SGD(model.parameters(), lr = lr)

In [237]:
model, opt = get_model()
loss_func( model(xb), yb)

tensor(2.31, grad_fn=<NllLossBackward0>)

In [238]:
for epoch in range(epochs):
    for i in range(0, n, bs):
        s = slice(i, min(n,i+bs))
        xb,yb = x_train[s],y_train[s]
        preds = model(xb)
        loss = loss_func(preds, yb)
        loss.backward()
        opt.step()
        opt.zero_grad()
    report(loss, preds, yb)



0.15, 0.92
0.11, 0.96
0.07, 0.96


# Dataset and DataLoader


## Dataset
  It's clunky to iterate through minibatches of x and y values separately:
  
  
* `xb,yb = x_train[s],y_train[s]`
  instead let's do both steps together
  `xb, yb = train_ds[s]`


In [239]:
class Dataset:
  def __init__( self,x,y):
    self.x, self.y = s,y

  def __len__(self): return len(self.x)

  def __getitem__(self, i): return self.x[i], self.y[i]

In [238]:
train_ds, valid_ds = Dataset(x_train)